In [3]:
#Ho scaricato il dizionario delle collocazioni da qui https://downloads.freemdict.com/%E5%B0%9A%E6%9C%AA%E6%95%B4%E7%90%86/%E5%85%B1%E4%BA%AB2020.5.11/content/4_others/italian/Dizionario%20delle%20collocazioni%20Le%20combinazioni%20delle%20parole%20in%20italiano/
#Utilizzato questo tool per convertirlo da .mdx a .json, https://github.com/ilius/pyglossary/tree/master

import json
import re
from bs4 import BeautifulSoup
from collections import defaultdict

#Ripuliamo le entrate del dizionario
STOPWORDS = {
    "a", "ad", "al", "allo", "ai", "agli", "all", "alla", "alle", "ha", "fa", "con", "col", "coi", "v", "verbi", "è", "nm", "f", 
    "all'", "e", "dà", "da", "dal", "dallo", "dai", "dagli", "dall", "dall'", "dalla", "dalle", "di", "del", "dello", "dei", 
    "degli", "dell", "dell'", "della", "delle", "in", "nel", "nello", "nei", "negli", "nell", "nell'", "nella", "nelle", "su", 
    "sul", "sullo", "sui", "sugli", "sull", "sull'", "sulla", "sulle", "dell'", "per", "tra", "fra", "avverbi", "pl", "ns", "s", 
    "aggettivo", "più", "aggettivi", "verbo", "agg", "qlco", "qlcu", "all", "si", "inv", "non", "ecc", "il", "lo", "la", "i", 
    "gli", "le", "l", "un", "uno", "una", "un'", "et", "na", "suo", "sua", "e", "ed", "o", 
    "od", "ma", "né", "ne", "che", "se", "quando", "come", "quale", "quali", "cui", "nf", "all'", "complemento", "complementi", 
    "AVVERBI", "AGGETTIVI", "VERBO", "COMPLEMENTO", "SOGGETTO", "soggetto"
}

def clean_entry(lemma, raw_html):
    #Rimuovo i <link>
    raw_html = re.sub(r'<link[^>]+>', '', raw_html)

    # arsing HTML
    soup = BeautifulSoup(raw_html, "html.parser")
    text = soup.get_text(separator=" ")

    #Trovo solo parole e locuzioni
    words = re.findall(r"[a-zA-ZàèéìòùÀÈÉÌÒÙ']+", text)

    #Pulizia
    cleaned = []
    for w in words:
        wl = w.lower()
        #Ignoro lo stesso lemma, le stopwords e tutte le parole
        #con lunghezza <= 2
        if wl == lemma.lower() or wl in STOPWORDS or len(wl) <= 2 or len(wl) >= 10:
            continue
        cleaned.append(wl)
    return cleaned

#Carico il JSON di partenza
with open("dizionari/dizionario.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#Creo un dizionario unificato
cleaned_dict = defaultdict(list)

for lemma, html in data.items():
    #Salto chiavi che contengono ##
    if "##" in lemma:
        continue

    #Rimuovo eventuali numeri tipo mente(2) → mente
    base_lemma = re.sub(r'\(\d+\)', '', lemma)

    cleaned_words = clean_entry(base_lemma, html)

    # Unifico tutte le parole dello stesso lemma
    cleaned_dict[base_lemma].extend(cleaned_words)

#Rimuovo duplicati
cleaned_dict = {k: list(set(v)) for k, v in cleaned_dict.items()}

#Salvo il risultato
with open("dizionari/dizionario_pulito.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_dict, f, ensure_ascii=False, indent=2)

print("Prime 10 parole del dizionario pulito:")
for i, word in enumerate(list(cleaned_dict.keys())[:10], start=1):
    print(f"{i}. {word}")



Prime 10 parole del dizionario pulito:
1. abbagliare
2. abbaglio
3. abbaiare
4. abbandonare
5. abbandonarsi
6. abbandono
7. abbassamento
8. abbassare
9. abbattere
10. abbigliamento


In [10]:
#Osservando il file .json però mi sono accorto che tutte le parole dovevano essere
#collegate biunivocamente, in quanto solitamente per gli autori del gioco
#nervi->saldi oppure saldi-nervi hanno lo stesso valore.

#Carica il dizionario pulito
with open("dizionari/dizionario_pulito.json", "r", encoding="utf-8") as f:
    data = json.load(f)

#Dizionario aumentato
augmented = defaultdict(list)

for lemma, words in data.items():
    #aggiungo la relazione originale
    augmented[lemma].extend(words)
    
    #creo le relazioni inverse
    for w in words:
        if lemma not in augmented[w]:  #evita duplicati
            augmented[w].append(lemma)

#Rimuovo duplicati in ogni lista
augmented = {k: list(set(v)) for k, v in augmented.items()}

#Salvo il risultato
with open("dizionari/dizionario_b.json", "w", encoding="utf-8") as f:
    json.dump(augmented, f, ensure_ascii=False, indent=2)

print("Prime 10 parole del dizionario:")
for i, word in enumerate(list(augmented.keys())[:10], start=1):
    print(f"{i}. {word}")


Prime 10 parole del dizionario:
1. abbagliare
2. offuscare
3. illudere
4. vista
5. abbaglio
6. fatale
7. totale
8. lieve
9. solito
10. tremendo


In [4]:
import random

#Qui ho scritto una fuzione per valutare la bontà delle catene che si potrebbero generare
def normalize(word):
    word = word.lower()
    word = word.strip(",.!?;:'")  #Rimuove punteggiatura
    return word

def catene_logiche_no_dup(dizionario, n_catene=5, lunghezza_catena=6):
    catene_generati = []

    keys = list(dizionario.keys())

    for _ in range(n_catene):
        catena = []
        normalizzati = set()  #traccia dei lemmi già presenti
        current = random.choice(keys)
        catena.append(current)
        normalizzati.add(normalize(current))

        for _ in range(lunghezza_catena - 1):
            possibili = dizionario.get(current, [])
            #filtra parole che non sono già normalizzate nella catena
            possibili = [w for w in possibili if normalize(w) not in normalizzati]
            if not possibili:
                break
            next_word = random.choice(possibili)
            catena.append(next_word)
            normalizzati.add(normalize(next_word))

            #aggiorna current solo se next_word è chiave nel dizionario
            if next_word in dizionario:
                current = next_word
            else:
                break

        catene_generati.append(catena)

    #stampa catene
    for i, c in enumerate(catene_generati, 1):
        print(f"Catena {i}: {' -> '.join(c)}")

#Carica il JSON bidirezionale
with open("dizionari/dizionario_b.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(catene_logiche_no_dup(data, n_catene=5, lunghezza_catena=6))

#Le catene diciamo che hanno alti e bassi:
#i legami tra le parole ci sono sempre ma a volte per capirli
#bisogna rifletterci, quelli del programma televisivo sono molto
#più immediati

Catena 1: cuocere -> bene -> prezioso -> documento -> reperire -> materiale
Catena 2: case -> gruppo -> pressione -> sentire -> vocazione -> scoprire
Catena 3: medicina -> tollerare -> sopraffazione -> atto -> crudele -> sovrano
Catena 4: comunale -> imposta -> pagare -> cifra -> denaro -> usare
Catena 5: diocesano -> palazzo -> popolare -> canzone -> d'autore -> quadro
None


In [6]:
import json
import random

#Carica il JSON
with open("dizionari/dizionario_b.json", "r", encoding="utf-8") as f:
    data = json.load(f)

num_examples = 50000
train_examples = []

#Lista delle chiavi non vuote
valid_keys = [k for k, v in data.items() if v]  #solo chiavi con almeno una parola

for _ in range(num_examples):
    #Scelgo una chiave principale casuale tra quelle valide
    start_word = random.choice(valid_keys)
    
    #Scelgo una parola target casuale all'interno della lista
    target_word = random.choice(data[start_word])
    
    #Prendo la prima lettera del target
    first_letter = target_word[0]
    
    #Creo il testo con un token speciale (intendo utilizzare gpt2)
    text = f"{start_word} -> {first_letter} [ANSWER] {target_word}"
    
    train_examples.append({"text": text})

#Mostra le prime 10 voci del dataset
print("Prime 10 voci del dataset:")
for i, example in enumerate(train_examples[:10]):
    print(f"{i+1}: {example['text']}")

#Salva in un file JSON
with open("dizionari/generated_dataset.json", "w", encoding="utf-8") as f:
    json.dump(train_examples, f, ensure_ascii=False, indent=2)


Prime 10 voci del dataset:
1: incrinare -> p [ANSWER] pacatezza
2: lucido -> p [ANSWER] profilo
3: accanimento -> t [ANSWER] testardo
4: regressione -> p [ANSWER] profonda
5: incontro -> f [ANSWER] fiasco
6: cellulare -> a [ANSWER] ammasso
7: disoccupazione -> r [ANSWER] ridurre
8: irresponsabile -> v [ANSWER] veramente
9: internet -> c [ANSWER] collegamento
10: dogana -> b [ANSWER] bloccare


In [ ]:
from datasets import Dataset                   
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model    
import torch                                  

dataset = Dataset.from_list(train_examples)  

dataset = dataset.train_test_split(test_size=0.2, seed=42)

#Carica tokenizer preaddestrato per distilgpt2
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")

#Aggiunge token speciale [ANSWER] che useremo per separare input e output nel training
special_tokens_dict = {'additional_special_tokens': ['[ANSWER]']}
tokenizer.add_special_tokens(special_tokens_dict)

#Imposta il token di padding uguale a eos_token perché GPT2 non ha un pad_token 
tokenizer.pad_token = tokenizer.eos_token  


#Carica modello pre-addestrato distilgpt2
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

#Aggiorna la dimensione del vocabolario del modello per includere i token speciali
model.resize_token_embeddings(len(tokenizer))

#Assicura che il modello sappia quale token usare come pad
model.config.pad_token_id = tokenizer.pad_token_id

lora_config = LoraConfig(
    r=8,                 
    lora_alpha=32,        
    target_modules=["c_attn"],  # moduli GPT2 dove applicare LoRA
    lora_dropout=0.1,     
    bias="none",          
    task_type="CAUSAL_LM" 
)

#Applica LoRA al modello
model = get_peft_model(model, lora_config)

def tokenize(batch):
    """
    Tokenizza batch di testi, tronca o riempie a max_length=32.
    Copia input_ids in labels per il Language Modeling.
    """
    encoding = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=32
    )
    encoding["labels"] = encoding["input_ids"].copy()  # etichette = input_ids
    return encoding

#Applica la tokenizzazione a tutto il dataset
dataset = dataset.map(tokenize, batched=True)


training_args = TrainingArguments(
    output_dir="./results",            
    overwrite_output_dir=True,         
    eval_strategy="epoch",             
    learning_rate=2e-4,                
    per_device_train_batch_size=8,     
    per_device_eval_batch_size=8,
    num_train_epochs=3,                
    weight_decay=0.01,                
    logging_steps=50,                  
    save_strategy="epoch"             
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],   
    eval_dataset=dataset["test"],     
    tokenizer=tokenizer
)

trainer.train()                        

save_path = "./finetuned-distilgpt2-lora"
trainer.save_model(save_path)          #salva modello fine-tuned
tokenizer.save_pretrained(save_path)   #salva tokenizer aggiornato


model = GPT2LMHeadModel.from_pretrained(save_path)
tokenizer = GPT2Tokenizer.from_pretrained(save_path)

prompt = "cane -> l [ANSWER]"         #prompt di esempio
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_new_tokens=3,                 
    num_return_sequences=3,           #tre alternative
    do_sample=False,                    #sampling casuale disattivato
    top_k=50,                          #considera solo i top 50 token più probabili
    temperature=1                   #controlla la "creatività" (+alto +creativo)
)

# Stampa i risultati
print("\nPrompt:", prompt)
print("Tre predizioni candidate:")
for i, output in enumerate(outputs):
    text = tokenizer.decode(output, skip_special_tokens=True)
    pred = text.replace(prompt, "").strip()
    print(f"{i+1}) {pred}")


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

/tmp/ipykernel_120439/1230961968.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/home/cosimo/anaconda3/envs/chain/lib/python3.12/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
save_path = "./finetuned-distilgpt2"

print(f"🔄 Carico modello da {save_path}...")
model = GPT2LMHeadModel.from_pretrained(save_path)
tokenizer = GPT2Tokenizer.from_pretrained(save_path)
print("✅ Modello caricato!\n")

def generate_predictions(prompt, num_predictions=5):
    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=4,          # genera fino a 3 token per completare
        num_return_sequences=num_predictions, 
        do_sample=True,            #sampling casuale
        top_k=50,                  #scegli tra i migliori 50
        repetition_penalty=3.0,      #penalità per la ripetizione
        temperature=0.9            #regola la diversità
    )

    predictions = []
    for output in outputs:
        text = tokenizer.decode(output, skip_special_tokens=True)
        pred = text.replace(prompt, "").strip()
        predictions.append(pred)
    return predictions

print("💬 Modalità interattiva: inserisci una parola e poi l'iniziale successiva.")
print("   Esempio: parola = cane, iniziale = l -> cane -> l [ANSWER]")
print("   Digita 'exit' in qualunque momento per uscire.\n")

while True:
    #Primo input: parola
    first_word = input("👉 Inserisci la prima parola: ").strip()
    if first_word.lower() == "exit":
        print("👋 Uscita dal programma.")
        break

    #Secondo input: iniziale della parola target
    initial = input("👉 Inserisci l'iniziale della parola successiva: ").strip()
    if initial.lower() == "exit":
        print("👋 Uscita dal programma.")
        break

    #Costruisci il prompt
    prompt = f"{first_word} -> {initial} [ANSWER]"

    #Genera predizioni
    preds = generate_predictions(prompt, num_predictions=5)

    #Stampa i risultati
    print(f"\n🔮 Predizioni per: {first_word} -> {initial}")
    for i, p in enumerate(preds, 1):
        print(f"{i}) {p}")
    print()


🔄 Carico modello da ./finetuned-distilgpt2...
✅ Modello caricato!

💬 Modalità interattiva: inserisci una parola e poi l'iniziale successiva.
   Esempio: parola = cane, iniziale = l -> cane -> l [ANSWER]
   Digita 'exit' in qualunque momento per uscire.



👉 Inserisci la prima parola:  occhi
👉 Inserisci l'iniziale della parola successiva:  l



🔮 Predizioni per: occhi -> l
1) occhi -> l  legito
2) occhi -> l  leggia
3) occhi -> l  letturo
4) occhi -> l  lamponte
5) occhi -> l  lato



KeyboardInterrupt: Interrupted by user